# *Text Generation using LSTM and Keras*

## Functions for Processing Text

### Reading in files as a string text

In [ ]:
!pip install tensorflow==2.5
#for this project, use tensorflow=2.5, because generate_new_text has predict_class which is contained only in 2.5

In [ ]:
def read_file(filepath):
    with open(filepath) as f:
        str_text=f.read()
    return str_text

In [ ]:
read_file('moby_dick_four_chapters.txt')

### Tokenize and Clean Text

In [ ]:

import spacy

In [ ]:
nlp=spacy.load('en_core_web_sm',disable=['parser','tagger','ner'])

In [ ]:
nlp.max_length=1198623


In [ ]:
def separate_punc(doc_test):
    return [token.text.lower() for token in nlp(doc_test) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ' ]

In [ ]:
d=read_file('moby_dick_four_chapters.txt')

In [ ]:
tokens=separate_punc(d)

In [ ]:
tokens

In [ ]:
len(tokens)

## Create Sequences of Tokens

In [ ]:

train_len=25+1
text_sequence=[]  #empty list
for i in range(train_len,len(tokens)):
    seq=tokens[i-train_len:i]
    text_sequence.append(seq)

In [ ]:
type(text_sequence)

In [ ]:
text_sequence[0]

In [ ]:
text_sequence[1]

In [ ]:
' '.join(text_sequence[0])

In [ ]:
' '.join(text_sequence[1])

In [ ]:
' '.join(text_sequence[2])

### Keras Tokenization

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(text_sequence)

In [ ]:
sequences=tokenizer.texts_to_sequences(text_sequence) #converts to sequences

In [ ]:
sequences[0]

In [ ]:
sequences[1]

In [ ]:
tokenizer.index_word

In [ ]:
for i in sequences[0]:
    print(f"{i}:{tokenizer.index_word[i]}")

In [ ]:
tokenizer.word_counts

In [ ]:
vocabulary_size=len(tokenizer.word_counts)
vocabulary_size

In [ ]:
type(sequences) 

### Convert to Numpy Matrix

In [ ]:
import numpy as np


In [ ]:
sequences=np.array(sequences)
sequences

# Creat LSTM model

In [ ]:
#split the data into features and labels 
#1] X features (first n words of sequences)
#2] Y label (Next word after the sequence)
#fit the model 
#split will break first columns as features and last column as target we want to predict

from tensorflow.keras.utils import to_categorical


In [ ]:
X=sequences[:,:-1]

In [ ]:
y=sequences[:,-1]

In [ ]:
y=to_categorical(y,num_classes=vocabulary_size+1)

In [ ]:
y

In [ ]:
seq_len=X.shape[1]


In [ ]:
X.shape

In [ ]:
#to create model
from tensorflow.keras.models import Sequential

In [ ]:
from tensorflow.keras.layers import Dense,LSTM,Embedding

In [ ]:
def create_model(vocabulary_size,seq_len):
    model=Sequential()
    model.add(Embedding(vocabulary_size,seq_len,input_length=seq_len))
    model.add(LSTM(50,return_sequences=True))
    model.add(LSTM(50)) #another layer of LSTM
    model.add(Dense(50,activation="relu"))
    model.add(Dense(vocabulary_size,activation="softmax")) #because it end, it will give words
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy']) #treat each vocabular word as an individual category
    model.summary()
    return model

In [ ]:
model=create_model(vocabulary_size+1,seq_len)

### Training the Model

In [ ]:
from pickle import dump,load 

In [ ]:
model.fit(X,y,batch_size=128,epochs=2,verbose=1)

In [ ]:
model.save('my_model.h5')  #save the model

In [ ]:

dump(tokenizer,open('my_tokenizer','wb'))

# generate new Text Based off a seed 

In [ ]:
from  tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:

def generate_new_text(model,tokenizer,seq_len,seed_text,num_gen_works):
    output_text=[]
    input_text=seed_text 
    for i in range(num_gen_works):
        encoded_text=tokenizer.texts_to_sequences([input_text])[0] 
        pad_encoded=pad_sequences([encoded_text],maxlen=seq_len,truncating='pre') 
        pred_word_ind=model.predict_classes(pad_encoded,verbose=0)[0]
        pred_word=tokenizer.index_word[pred_word_ind]
        input_text+=' '+pred_word
        output_text.append(pred_word)
    return ' '.join(output_text)

In [ ]:
text_sequence[0]

### Grab a random seed sequence

In [ ]:
import random
random.seed(101)
random_pick=random.randint(0,len(text_sequence))

In [ ]:
random_seed_text=text_sequence[random_pick]
random_seed_text

In [ ]:
seed_text=' '.join(random_seed_text)
seed_text

In [ ]:
generate_new_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_works=25)

#loading another model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model_fin=load_model('epochBIG.h5')

In [ ]:
tokenizer=load(open("epochBIG","rb"))


In [ ]:
generate_new_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_works=25)